In [1]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division

## download

In [2]:
# create --post-file for download
DIR = '/scratch/lindb/testdata/readSetLinks'
tmpfile = op.join(DIR,'.tmp.dat')
if not op.exists(tmpfile):
    with open(tmpfile,'w') as o:
        o.write("%s" % 'j_username=mlu1&j_password=rrgmmngu')
    !chmod 600 $tmpfile

In [3]:
# create dirs, open file with weblinks to fastq files
shdir = op.join(DIR,'shfiles')
if not op.exists(shdir):
    os.makedirs(shdir)
links = open(op.join(DIR,'readSetLinks.txt'),'r').readlines()
print len(links)
# make sure file name parse is correct
links[0].split("/")[-1].replace("\n","")

384


'HI.4779.007.D707---D508.JP_101_14_cap5_kit1_R1.fastq.gz'

In [ ]:
# get link dict to download both fastq and md5 file


In [6]:
# wget links
files = [f for f in fs('/home/lindb/scratch/testdata') if 'fastq' in f]
for i,link in enumerate(links):
    link = link.replace("\n","")
    fqgz = link.split("/")[-1]
    f = op.join('/home/lindb/scratch/testdata',fqgz)
    if not f in files: # faster to check if f in list than op.exists(f)
        text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=link%s
#SBATCH --export=all
#SBATCH --time=00:59:00
#SBATCH --mem=500mb
#SBATCH --cpus-per-task=1

cd /home/lindb/scratch/testdata
wget --no-cookies --no-check-certificate -c %s --post-file %s
''' % (str(i).zfill(3),
       link,
       tmpfile
      )
#         filE = op.join(shdir,'wget%s.sh' % str(i).zfill(3))
        filE = op.join(shdir,'wget%s.sh' % str(i).zfill(3))        
        with open(filE,'w') as o:
            o.write("%s" % text)

In [7]:
DIR = '/home/lindb/scratch/testdata/readSetLinks'
shdir = op.join(DIR,'shfiles')
fs(shdir)[0]

'/home/lindb/scratch/testdata/readSetLinks/shfiles/wget000.sh'

In [22]:
fs(shdir)[0]

'/home/lindb/scratch/testdata/readSetLinks/shfiles/wget000.sh'

In [3]:
os.getcwd()

'/home/lindb/scripts/coadaptree'

In [23]:
for f in fs(shdir):
    !sbatch $f

Submitted batch job 11049502
Submitted batch job 11049505
Submitted batch job 11049507
Submitted batch job 11049509
Submitted batch job 11049511
Submitted batch job 11049513
Submitted batch job 11049514
Submitted batch job 11049516
Submitted batch job 11049517
Submitted batch job 11049518
Submitted batch job 11049519
Submitted batch job 11049520
Submitted batch job 11049521
Submitted batch job 11049522
Submitted batch job 11049523
Submitted batch job 11049524
Submitted batch job 11049525
Submitted batch job 11049526
Submitted batch job 11049527
Submitted batch job 11049528
Submitted batch job 11049529
Submitted batch job 11049530
Submitted batch job 11049531
Submitted batch job 11049533
Submitted batch job 11049534
Submitted batch job 11049535
Submitted batch job 11049536
Submitted batch job 11049537
Submitted batch job 11049538
Submitted batch job 11049540
Submitted batch job 11049541
Submitted batch job 11049542
Submitted batch job 11049546
Submitted batch job 11049547
Submitted batc

Submitted batch job 11050033
Submitted batch job 11050034
Submitted batch job 11050036
Submitted batch job 11050037
Submitted batch job 11050038
Submitted batch job 11050040
Submitted batch job 11050042
Submitted batch job 11050043
Submitted batch job 11050045
Submitted batch job 11050046
Submitted batch job 11050047
Submitted batch job 11050049
Submitted batch job 11050050
Submitted batch job 11050055
Submitted batch job 11050062
Submitted batch job 11050065
Submitted batch job 11050069
Submitted batch job 11050073
Submitted batch job 11050074
Submitted batch job 11050075
Submitted batch job 11050079
Submitted batch job 11050082
Submitted batch job 11050084
Submitted batch job 11050086
Submitted batch job 11050089
Submitted batch job 11050091
Submitted batch job 11050092
Submitted batch job 11050096
Submitted batch job 11050098
Submitted batch job 11050100
Submitted batch job 11050101
Submitted batch job 11050104
Submitted batch job 11050106
Submitted batch job 11050109
Submitted batc

In [8]:
# or sbatch with master
text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=wgetmaster
#SBATCH --export=all
#SBATCH --time=00:59:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1


'''
for f in fs(shdir):
    newtext = '''sbatch %s
''' % f
    text = text + newtext
filE=op.join(shdir,'master.sh')
with open(filE,'w') as o:
    o.write("%s" % text)

In [9]:
# !sbatch filE

Submitted batch job 11049217


## check md5

In [20]:
# log to logfile
text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=md5sum
#SBATCH --export=all
#SBATCH --time=03:00:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1


cd /home/lindb/scratch/testdata/
md5sum -c *md5 > md5log.txt
'''
filE = op.join(shdir,'md5sumcheck.sh')
print filE
with open(filE,'w') as o:
    o.write("%s" % text)
!sbatch $fiLE

/home/lindb/scratch/testdata/readSetLinks/shfiles/md5sumcheck.sh
^C


In [1]:
# read the log, look for bad md5 checks
md5 = open('/home/lindb/scratch/testdata/md5log.txt').readlines()
for line in md5:
    res = line.split(" ")[1].split("\n")[0]
    if not res == "OK":
        print line
# one res in testdata was not ok, but I had gunzip-ed it

HI.4779.005.D708---D504.LP_mg13_cap3_kit1_R1.fastq.gz: FAILED



## split files by library

In [8]:
from Bio import SeqIO

In [16]:
# get a list of fastq R1 files
files = sorted([f for f in fs('/scratch/lindb/testdata') if 'R1' in f and f.endswith('fastq.gz')])
len(files)

96

In [12]:
# import info sheet
summary = pd.read_csv('/scratch/lindb/testdata/1_CoAdapTree_Seq_Summary.txt',sep='\t')
summary.head()

,Pool_name,Sample_name,Ploidy,Index1,Index2,Adaptor_1,Adaptor_2
0,LP_mg_cap2_kit1,LP_mg8_cap2_kit1,1,ATTACTCG,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
1,LP_mg_cap2_kit1,LP_mg22_cap2_kit1,1,TCCGGAG,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
2,LP_mg_cap2_kit1,LP_mg64_cap2_kit1,1,GAATTCGT,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
3,LP_mg_cap2_kit1,LP_mg9_cap2_kit1,1,CGCTCATT,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
4,LP_mg_cap2_kit1,LP_mg10_cap2_kit1,1,GAGATTCC,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT


In [13]:
samps = sorted(summary['Sample_name'].tolist())
len(samps)

96

In [14]:
# fill in blank pool names, create dictionary for samps in pools, get a list of spp, get rg info
sampdict = {}
spp      = []
for row in summary.index:
    if type(summary.loc[row,'Pool_name']) == float:
        summary.loc[row,'Pool_name'] = summary.loc[row,'Sample_name']
    pool = summary.loc[row,'Pool_name']
    samp = summary.loc[row,'Sample_name']
    sp   = pool.split("_")[0]
    if not sp in spp:
        spp.append(sp)
        spp = sorted(spp)
    sampdict[samp] = pool


In [15]:
# get rg info
rg = '/scratch/lindb/mengmeng/4_addRG.sh'
rg = open(rg,'r').readlines()
rginfo = {}
for line in rg:
    if line.split(" ")[0] == 'java':
        splits = line.split(" ")
        samp = splits[12].split("/")[-1].split(".")[0]
        assert samp in sampdict
        if not samp in rginfo:
            rginfo[samp] = {}
        else:
            print 'duplicate samp %s' % samp
        rginfo[samp]['rgid'] = splits[6].split("=")[1]
        rginfo[samp]['rglb'] = splits[7].split("=")[1]
        rginfo[samp]['rgpl'] = splits[9].split("=")[1]
        rginfo[samp]['rgpu'] = splits[10].split("=")[1]
        rginfo[samp]['rgsm'] = splits[11].split("=")[1]
        #print samp,rginfo[samp]

In [16]:
len(rginfo.keys()) == len(sampdict.keys())

True

In [17]:
pkldump(rginfo,'/scratch/lindb/testdata/rginfo.pkl')

In [18]:
# list of files by pool
poolfdict = {}
ftopool = {}
for f in files:
    samp = f.split("---")[1].split(".")[1].replace("_R1","")
    pool = sampdict[samp]
    if not pool in poolfdict:
        poolfdict[pool] = []
    f2 = op.abspath(f).replace("_R1","_R2")
    assert op.exists(f2)
    [poolfdict[pool].append(x) for x in [f,f2]]
    ftopool[f]  = pool
    ftopool[f2] = pool
    
# poolfdict

In [19]:
samp = sampdict.keys()[0]
sampdict[samp]

'JP_cap4_kit1'

# something else

In [22]:
## kick off lodgepole and jack pine
jplp = [d for d in dirs if 'LP' in d or 'JP' in d]
jplp

['/scratch/lindb/testdata/JP_cap3_kit1',
 '/scratch/lindb/testdata/JP_cap4_kit1',
 '/scratch/lindb/testdata/JP_cap5_kit1',
 '/scratch/lindb/testdata/JP_p101_cap1_kit1',
 '/scratch/lindb/testdata/JP_p102_cap2_kit1',
 '/scratch/lindb/testdata/LP_mg_cap2_kit1',
 '/scratch/lindb/testdata/LP_mg_cap3_kit1',
 '/scratch/lindb/testdata/LP_mg_cap4_kit1',
 '/scratch/lindb/testdata/LP_mg_cap5_kit1',
 '/scratch/lindb/testdata/LP_mg_cap6_kit1',
 '/scratch/lindb/testdata/LP_pmg40_cap1_kit1']

In [23]:
# get nfiles per dir
fcount = 0
dcount = {}
for d in jplp:
    fcount += len(ls(d))
    dcount[d] = len(ls(d))
fcount

134865

In [24]:
# determine frac of sh files
sums = sum(dcount.values())
frac = {}
for d in dcount:
    frac[d] = dcount[d]/sums
sum(frac.values()),frac

(1.0,
 {'/scratch/lindb/testdata/JP_cap3_kit1': 0.09340451562673785,
  '/scratch/lindb/testdata/JP_cap4_kit1': 0.09445000556111667,
  '/scratch/lindb/testdata/JP_cap5_kit1': 0.10315500685871057,
  '/scratch/lindb/testdata/JP_p101_cap1_kit1': 0.11344678011344678,
  '/scratch/lindb/testdata/JP_p102_cap2_kit1': 0.09496904311719126,
  '/scratch/lindb/testdata/LP_mg_cap2_kit1': 0.09909168427686946,
  '/scratch/lindb/testdata/LP_mg_cap3_kit1': 0.0905942980017054,
  '/scratch/lindb/testdata/LP_mg_cap4_kit1': 0.07270974678382086,
  '/scratch/lindb/testdata/LP_mg_cap5_kit1': 0.06676305935565195,
  '/scratch/lindb/testdata/LP_mg_cap6_kit1': 0.07742557372187002,
  '/scratch/lindb/testdata/LP_pmg40_cap1_kit1': 0.09399028658287918})

In [25]:
# num sh files per dir
shfrac = {}
for d in frac:
    shfrac[d] = int(math.ceil(frac[d]*975))
    print d, shfrac[d]

/scratch/lindb/testdata/LP_mg_cap5_kit1 66
/scratch/lindb/testdata/LP_mg_cap4_kit1 71
/scratch/lindb/testdata/LP_mg_cap3_kit1 89
/scratch/lindb/testdata/LP_mg_cap6_kit1 76
/scratch/lindb/testdata/JP_cap3_kit1 92
/scratch/lindb/testdata/LP_mg_cap2_kit1 97
/scratch/lindb/testdata/LP_pmg40_cap1_kit1 92
/scratch/lindb/testdata/JP_p101_cap1_kit1 111
/scratch/lindb/testdata/JP_p102_cap2_kit1 93
/scratch/lindb/testdata/JP_cap4_kit1 93
/scratch/lindb/testdata/JP_cap5_kit1 101


In [27]:
shdir = '/scratch/lindb/testdata/shfiles/start_trim'
if not op.exists(shdir):
    os.makedirs(shdir)
for d in shfrac:
    text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=%s
#SBATCH --export=all
#SBATCH --time=12:00:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1
#SBATCH --mail-user=brandon.lind@ubc.ca
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL


source $HOME/.bashrc
cd $HOME/pipeline

python 01a_trim-fastq.py %s %s %s
''' % (op.basename(d),
       d,
       '/scratch/lindb/refgenomes/LP_db/ptaeda.v1.01.reduced.pseudo.fasta',
       str(shfrac[d])
      )
    filE = op.join(shdir,'start-trim_%s.sh' % op.basename(d))
    with open(filE,'w') as o:
        o.write("%s" % text)

In [30]:
for f in fs(shdir):
    !sbatch $f

Submitted batch job 10834343
Submitted batch job 10834344
Submitted batch job 10834345
Submitted batch job 10834346
Submitted batch job 10834348
Submitted batch job 10834349
Submitted batch job 10834350
Submitted batch job 10834351
Submitted batch job 10834352
Submitted batch job 10834353


# why one command?

In [3]:
fqdir = '/scratch/lindb/testdata/JP_cap3_kit1/'

In [4]:
lensh = 92

In [5]:
gzfiles = [f for f in fs(fqdir) if 'R1' in f]
lgz     = len(gzfiles)
lgz

6297

In [32]:
seq_pairs = []
for f in gzfiles:
    assert 'R1' in f
    read2 = f.replace("_R1","_R2")
    assert op.exists(read2)
    if op.exists(read2):
        seq_pairs.append((f,read2))
#     else:
#         !echo 'no pair for '$f >> $fqdir'/messages/msgs.txt' # #iheartjupyter
    else:
        text = 'no pair for %s' % f
        with open(mfile,'a') as o:
            o.write("%s\n" % text)
print "found %s R1/R2 seq pairs" % str(len(seq_pairs))

found 6297 R1/R2 seq pairs


In [7]:
if len(seq_pairs) <= lensh:
    # one command per sh file
    ceil = 1
else:
    # multiple commands per sh file
    ceil = math.ceil(len(seq_pairs)/lensh)
ceil

69.0

In [17]:
lensh

92

In [12]:
shcount = 0
fcount  = 0
tcount  = 0
text = ''''''
for s in seq_pairs:
#     print s
    r1    = op.abspath(s[0])
    if r1.endswith("fastq"):
        r1out = 'hey'
    else:
        r1out = 'hey'
    r2    = op.abspath(s[1])
    if r2.endswith("fastq"):
        r2out = 'hey'
    else:
        r2out = 'hey'
    html = 'hey'
    json = 'hey'
    cmd  = '''fastp -i %s -o %s -I %s -O %s -g --cut_window_size 5 --cut_mean_quality 30 --qualified_quality_phred 30 --unqualified_percent_limit 20 --n_base_limit 5 --length_required 75 -h %s.html --cut_by_quality3 --thread 16 --json %s.json --adapter_sequence AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC --adapter_sequence_r2 AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT


cd $HOME/pipeline
# once finished, map using bwa mem 
python 01b_bwa-map.py %s %s %s %s %s           
''' % (r1  , r1out,
       r2  , r2out,
       html, json ,
       'hi' , r1out,  r2out,  'shdir',  str(tcount).zfill(3)
      )
    text = text + cmd
    
    fcount += 1
    tcount += 1
    if fcount == ceil or tcount == len(seq_pairs):
        shz = str(shcount).zfill(3)
        shcount += 1
        fcount = 0
        text = ''''''

In [13]:
shcount

92

In [20]:
len([x for x in ls('/scratch/lindb/testdata/JP_cap4_kit1/shfiles/trimmed_shfiles/') if 'sh' in x])

6369

In [45]:
DIR = '/scratch/lindb/testdata/'
os.chdir(DIR)
dirs = !ls -d *kit*/
dirs = [op.abspath(d) for d in dirs if 'JP' in d or 'LP' in d]
dirs

['/scratch/lindb/testdata/JP_cap3_kit1',
 '/scratch/lindb/testdata/JP_cap4_kit1',
 '/scratch/lindb/testdata/JP_cap5_kit1',
 '/scratch/lindb/testdata/JP_p101_cap1_kit1',
 '/scratch/lindb/testdata/JP_p102_cap2_kit1',
 '/scratch/lindb/testdata/LP_mg_cap2_kit1',
 '/scratch/lindb/testdata/LP_mg_cap3_kit1',
 '/scratch/lindb/testdata/LP_mg_cap4_kit1',
 '/scratch/lindb/testdata/LP_mg_cap5_kit1',
 '/scratch/lindb/testdata/LP_mg_cap6_kit1',
 '/scratch/lindb/testdata/LP_pmg40_cap1_kit1']

In [49]:
for d in dirs:
    print d
    os.chdir(d)
    !rm -r -- ./*/

/scratch/lindb/testdata/JP_cap3_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/JP_cap4_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/JP_cap5_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/JP_p101_cap1_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/JP_p102_cap2_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_mg_cap2_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_mg_cap3_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_mg_cap4_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_mg_cap5_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_mg_cap6_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_pmg40_cap1_kit1
rm: cannot remove './*/': No such file or 